In [1]:
def fd(s): # замена цен на разницу цен
    for i in range(np.size(s)-1):
        s[i] = s[i+1]/s[i] - 1

def profitability_and_risk(s): # поиск среднего риска и средней доходности 
    fd(s)
    s = s[:11]
    smean = np.mean(s)
    average_income.append(smean)
    sstd = np.std(s)
    average_risk.append(sstd)

def total_revenue_function(p, sd): # поиск общей доходности 
    od = 0
    for i in range(np.size(p)):
        od +=  p[i]*sd[i]
    return od


import numpy as np
from scipy.optimize import minimize

# получение котировок из файла 
securities1 = np.genfromtxt('1.txt', delimiter=' ', names=True)
securities2 = np.genfromtxt('2.txt', delimiter=' ', names=True)
securities3 = np.genfromtxt('3.txt', delimiter=' ', names=True)

# начальные доли акций 
proportion1 = 0.25 
proportion2 = 0.25 
proportion3 = 0.5
proportion = np.array([proportion1, proportion2, proportion3])

# средняя доходность и средний риск
average_income = []
average_risk = []

# избавление от лишних столбцов
securities1 = securities1['CLOSE']
securities2 = securities2['CLOSE']
securities3 = securities3['CLOSE']

# преобразование основного массива к нужному виду 
tmp = np.array([securities1, securities2, securities3])
for i in range(len(tmp)):
    profitability_and_risk(tmp[i])
for i in range(3):
    tmp[i] = tmp[i][:11]
    
array = np.zeros((3,11))
for i in range(3):
    array[i] = tmp[i][:11]

# роиск ковариационной матрицы
cov = np.cov(array)

# поиск общего риска портфеля (есть ошибка!)
total_risk = np.matmul(proportion, cov)
total_risk = np.matmul(total_risk, proportion)

# поиск общей доходности портфеля
total_revenue = total_revenue_function(proportion, average_income)

In [2]:
# Вывод данных о портфеле
print("Ковариационная матрица ", '\n', cov)
print('\n')
for i in range(3):
    print("Доходность акции ", i+1, ":", "%.2f" % (average_income[i]*100), "%")
print('\n')
for i in range(3):
    print("Риск акции ", i+1, ":", "%.2f" % (average_risk[i]*100), "%")
print('\n')
print("Общяя доходность портфеля ", "%.2f" % (total_revenue*100), "%")
print("Общий риск портфеля ", "%.2f" % (total_risk*100), "%")

Ковариационная матрица  
 [[ 0.00111939 -0.00093815  0.00038052]
 [-0.00093815  0.01294495 -0.00145022]
 [ 0.00038052 -0.00145022  0.00426609]]


Доходность акции  1 : 3.11 %
Доходность акции  2 : 4.34 %
Доходность акции  3 : 2.58 %


Риск акции  1 : 3.19 %
Риск акции  2 : 10.85 %
Риск акции  3 : 6.23 %


Общяя доходность портфеля  3.15 %
Общий риск портфеля  0.16 %


In [3]:
# Минимизация риска при заданной доходности в 2% (не работает!)
def objective(x, cov, sr): # функция риска
         x1=x[0];x2=x[1];x3=x[2]
         return x1**2*sr[0]**2+x1*x2*cov[0][1]+x1*x3*cov[0][2]+x2*x1*cov[1][0]+x2**2*sr[1]**2+x2*x3*cov[1][2]+x3*x1*cov[2][0]+x3*x2*cov[2][1]+x3**2+sr[2]**2
        
def constraint1(x): # ограничение на сумму долей
         return x[0]+x[1]+x[2]-1.0
def constraint2(x): # задание конкретной доходности       
         return average_income[0]*x[0] + average_income[1]*x[1] + average_income[2]*x[2] - 0.02
    
x0=[proportion1,proportion2,proportion3]# изначальные доли акций

b=(0.0,1.0) # диапозон X
bnds=(b,b,b)

con1={'type':'ineq','fun': constraint1}
con2={'type':'eq','fun': constraint2}
cons=[con1,con2]

sol=minimize(objective,x0,args=(cov,average_risk),method='SLSQP',bounds=bnds,constraints=cons)
sol

     fun: 1.003878267268145
     jac: array([ 7.61047006e-04, -2.90043652e-03,  1.99999999e+00])
 message: 'Positive directional derivative for linesearch'
    nfev: 168
     nit: 20
    njev: 16
  status: 8
 success: False
       x: array([0.00000000e+00, 3.05436096e-15, 1.00000000e+00])